In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as T
import random

from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.optim import Adam, SGD, lr_scheduler
from torch.autograd import Variable

from random import randint, shuffle

from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def imageshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def add_noise(inputs, noise_factor=0.1):
     noisy = inputs+torch.randn_like(inputs) * noise_factor
     noisy = torch.clip(noisy,0.,1.)
     return noisy

def add_random_boxes(img,n_k,size=10):
    h,w = size,size
    img_size = img.shape[1]
    nparr = img.numpy()
    #boxes = []
    for _ in range(n_k):
        y,x = np.random.randint(0,img_size-w,(2,))
        for i in range(len(nparr)):
            nparr[i][y:y+h,x:x+w] = 0
        #boxes.append((x,y,h,w))
    return torch.from_numpy(nparr)

In [ ]:
torch.manual_seed(0)
img_size = 100

In [ ]:
# + black blocks tr
aug_t_f = [
    T.Grayscale(3),
    T.RandomRotation(180),
    T.GaussianBlur(33,1.1),
    add_noise,
    T.ColorJitter(
        (0.5,1.3),
        (0.5,1.3),
        (0.5,1.3),   
    ),
    lambda img: T.Resize(img_size)(T.RandomCrop(int(img_size*0.7))(img)),
    lambda img: add_random_boxes(img, randint(1,5))
]
# + multiplie transforms
def random_transform(img):    
    n_tr = randint(1, len(aug_t_f))
    shuffle(aug_t_f)

    new_img = img
    for _, tr in zip(range(n_tr), aug_t_f):
        tr = random.choice(aug_t_f)
    new_img = tr(img)

    return new_img

In [ ]:
d_mean = (0.485, 0.456, 0.406)
t_std = (0.229, 0.224, 0.225)

# - normalize rem
transform = T.Compose([
    T.Resize(img_size),
    T.CenterCrop(img_size),
    T.ToTensor(),
    #T.Normalize(d_mean, t_std)
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

aug_transform = T.Compose([
    T.Resize(img_size),
    T.CenterCrop(img_size),
    T.ToTensor(),
    random_transform,
    #T.Normalize(d_mean, t_std)
])


In [ ]:
dataset_ = datasets.ImageFolder('training_set_t', transform=transform)
N_CLASSES = len(dataset_.classes)

aug_dataset = datasets.ImageFolder('training_set_t', transform=aug_transform)
dataset = dataset_ + aug_dataset


In [ ]:
img1_ = dataset_[-1][0]
img2_ = dataset[-1][0]
imageshow(img1_)
imageshow(img2_)

In [ ]:
train_set, val_set = random_split(dataset, [0.8, 0.2])
trainloader, valloader = DataLoader(train_set, batch_size=64, shuffle=True), DataLoader(val_set, batch_size=64, shuffle=True)

In [ ]:
"""dataset_ = dataset[0][0] / 255 #normalization of pixels
train_mean = dataset_.reshape(3,-1).mean(axis=1)
train_std = dataset_.reshape(3,-1).std(axis=1)
print(train_mean, train_std)"""

In [ ]:
class mush_rec_nn_old(nn.Module):
    drop_c = 0.4
    def __init__(self):
        super(mush_rec_nn_old, self).__init__()
        
        k_s = 2
        pad = 0

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=k_s, stride=1, padding=pad)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(12)
        
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=k_s, stride=1, padding=pad)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(24)
        
        self.cnn3 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=k_s, stride=1, padding=pad)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(48)
        
        self.cnn4 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=k_s, stride=1, padding=pad)
        self.pool4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(96)
        
        self.cnn5 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=k_s, stride=1, padding=pad)
        self.pool5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(192)
        
        linr = 192 * 2 * 2
        linr2 = linr
        self.fc1 = nn.Linear(linr, linr)
        self.fc2 = nn.Linear(linr, 192)
        self.dp1 = nn.Dropout(self.drop_c)
        self.fc3 = nn.Linear(192, N_CLASSES)
        self.sm = nn.Softmax(dim=1)
    
    def forward(self, input):
        output = self.pool1(F.relu(self.bn1(self.cnn1(input))))
        output = self.pool2(F.relu(self.bn2(self.cnn2(output))))  
        output = self.pool3(F.relu(self.bn3(self.cnn3(output))))
        output = self.pool4(F.relu(self.bn4(self.cnn4(output))))  
        output = self.pool5(F.relu(self.bn5(self.cnn5(output))))
        #print(output.shape)
        
        output = output.view(-1, 192 * 2 * 2)
        
        output = F.relu(self.fc1(output))
        output = self.fc2(output)
        output = self.dp1(output)
        output = self.fc3(output)
        output = self.sm(output)

        return output
N_CLASSES = 9
class mush_rec_nn(nn.Module):
    def __init__(self):
        super(mush_rec_nn, self).__init__()
        
        k_s = 3
        pad = 0

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=k_s, stride=1, padding=pad)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(24)
        
        self.cnn2 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=k_s, stride=1, padding=pad)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(48)
        
        self.cnn3 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=k_s, stride=1, padding=pad)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(96)
        
        self.cnn4 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=k_s, stride=1, padding=pad)
        self.pool4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(192)
        
        self.cnn5 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=k_s, stride=1, padding=pad)
        self.pool5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(384)
        
        linr = 384 * 1 * 1
        
        self.fc1 = nn.Linear(linr, linr)
        self.fc2 = nn.Linear(linr, 384)
        self.dp1 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(384, N_CLASSES)
    
    def forward(self, input):
        output = self.pool1(F.relu(self.bn1(self.cnn1(input))))
        output = self.pool2(F.relu(self.bn2(self.cnn2(output))))  
        output = self.pool3(F.relu(self.bn3(self.cnn3(output))))
        output = self.pool4(F.relu(self.bn4(self.cnn4(output))))  
        output = self.pool5(F.relu(self.bn5(self.cnn5(output))))
        
        output = output.view(-1, 384 * 1 * 1)
        
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.dp1(output)
        output = self.fc3(output)

        return output

class mush_rec_nn_adv(nn.Module):
    def __init__(self):
        super(mush_rec_nn, self).__init__()
        
        k_s = 3
        pad = 0

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=k_s, stride=1, padding=pad)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(24)
        
        self.cnn2 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=k_s, stride=1, padding=pad)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(48)
        
        self.cnn3 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=k_s, stride=1, padding=pad)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(96)
        
        self.cnn4 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=k_s, stride=1, padding=pad)
        self.pool4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(192)
        
        self.cnn5 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=k_s, stride=1, padding=pad)
        self.pool5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(384)
        
        linr = 384 * 1 * 1
        
        self.fc1 = nn.Linear(linr, linr)
        self.fc2 = nn.Linear(linr, 384)
        self.dp1 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(384, N_CLASSES)
    
    def forward(self, input):
        output = self.pool1(F.relu(self.bn1(self.cnn1(input))))
        output = self.pool2(F.relu(self.bn2(self.cnn2(output))))  
        output = self.pool3(F.relu(self.bn3(self.cnn3(output))))
        output = self.pool4(F.relu(self.bn4(self.cnn4(output))))  
        output = self.pool5(F.relu(self.bn5(self.cnn5(output))))
        
        output = output.view(-1, 384 * 1 * 1)
        
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.dp1(output)
        output = self.fc3(output)

        return output
    
model = mush_rec_nn()

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

In [ ]:
def saveModel():
    path = "./mush_rec_model_same.pth"
    torch.save(model.state_dict(), path)

def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    accuracy = (100 * accuracy / total)
    return(accuracy)

def train(num_epochs):
    global trainloader
    global trainloader, valloader
    
    best_accuracy = 0.0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    
    model.to(device)

    bad_a_c = 0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(trainloader, 0):
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            optimizer.zero_grad()
            
            outputs = model(images)
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            optimizer.step()

            running_loss += loss.item()
            if i % 1000 == 999:    
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                running_loss = 0.0

        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy
            model.dp1 = nn.Dropout(0.4)
            bad_a_c = 0
        else:
            bad_a_c += 1
            if bad_a_c >= 2:
                # + dp 0.8 -> 0.9
                 model.dp1 = nn.Dropout(0.9)
    
    train_set, val_set = random_split(dataset, [0.8, 0.2])
    trainloader, valloader = DataLoader(train_set, batch_size=64, shuffle=True), DataLoader(val_set, batch_size=64, shuffle=True)

In [ ]:
def testBatch():
    # get batch of images from the test DataLoader  
    images, labels = next(iter(valloader))

    # show all images as one image grid
    imageshow(make_grid(images))
   
    # Show the real labels on the screen 
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
                               for j in range(batch_size)))
  
    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)
    
    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)
    
    # Let's show the predicted labels on the screen to compare with the real ones
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
                              for j in range(batch_size)))

In [ ]:
train(55)
print('Finished Training')
# Test which classes performed well
testAccuracy()

# Let's load the model we just created and test the accuracy per label
model = mush_rec_nn()
path = "./mush_rec_model_same.pth"
model.load_state_dict(torch.load(path))
